In [2]:
source setup.tcl
set year 2016
set day 14
aoc::get-puzzle $year $day 1
aoc::get-puzzle $year $day 2

set input [string trim [aoc::get-input $year $day]];
jupyter::display text/plain [string range $input 0 100];




(cached)

--- Day 14: One-Time Pad --- In order to communicate securely with Santa while you're on this mission, you've been using a one-time pad that you generate using a pre-agreed algorithm . Unfortunately, you've run out of keys in your one-time pad, and so you need to generate some more. To generate keys, you first get a stream of random data by taking the MD5 of a pre-arranged salt (your puzzle input) and an increasing integer index (starting with 0 , and represented in decimal); the resulting MD5 hash should be represented as a string of lowercase hexadecimal digits. However, not all of these MD5 hashes are keys , and you need 64 new keys for your one-time pad. A hash is a key only if : 
 It contains three of the same character in a row, like 777 . Only consider the first such triplet in a hash. One of the next 1000 hashes in the stream contains that same character five times in a row, like 77777 . 
 Considering future hashes for five-of-a-kind sequences does not cause those hashes to be skipped; instead, regardless of whether the current hash is a key, always resume testing for keys starting with the very next hash. For example, if the pre-arranged salt is abc : 
 The first index which produces a triple is 18 , because the MD5 hash of abc18 contains ...cc38887a5... . However, index 18 does not count as a key for your one-time pad, because none of the next thousand hashes (index 19 through index 1018 ) contain 88888 . The next index which produces a triple is 39 ; the hash of abc39 contains eee . It is also the first key: one of the next thousand hashes (the one at index 816) contains eeeee . None of the next six triples are keys, but the one after that, at index 92 , is: it contains 999 and index 200 contains 99999 . Eventually, index 22728 meets all of the criteria to generate the 64 th key. 
 So, using our example salt of abc , index 22728 produces the 64 th key. Given the actual salt in your puzzle input, what index produces your 64 th one-time pad key?

(cached)

--- Part Two --- Of course, in order to make this process even more secure , you've also implemented key stretching . Key stretching forces attackers to spend more time generating hashes. Unfortunately, it forces everyone else to spend more time, too. To implement key stretching, whenever you generate a hash, before you use it, you first find the MD5 hash of that hash, then the MD5 hash of that hash, and so on, a total of 
 2016 additional hashings . Always use lowercase hexadecimal representations of hashes. For example, to find the stretched hash for index 0 and salt abc : 
 Find the MD5 hash of abc0 : 577571be4de9dcce85a041ba0410f29f . Then, find the MD5 hash of that hash: eec80a0c92dc8a0777c619d9bb51e910 . Then, find the MD5 hash of that hash: 16062ce768787384c81fe17a7a60c7e3 . ...repeat many times... Then, find the MD5 hash of that hash: a107ff634856bb300138cac6568c0f24 . 
 So, the stretched hash for index 0 in this situation is a107ff... . In the end, you find the original hash (one use of MD5), then find the hash-of-the-previous-hash 2016 times, for a total of 2017 uses of MD5. The rest of the process remains the same, but now the keys are entirely different. Again for salt abc : 
 The first triple ( 222 , at index 5 ) has no matching 22222 in the next thousand hashes. The second triple ( eee , at index 10 ) hash a matching eeeee at index 89 , and so it is the first key. Eventually, index 22551 produces the 64 th key (triple fff with matching fffff at index 22859 . 
 Given the actual salt in your puzzle input and using 2016 extra MD5 calls of key stretching, what index now produces your 64 th one-time pad key?

(cached)

ngcjuoqr

In [3]:
##timeit
package require md5
set seed $input
array unset cands
array unset hashes
set keys {}

set index 0
for {set index 0} {$index < 50000 } {incr index} {
    set hashes($index) [md5::md5 -hex $seed$index]
    if {[regexp {(.)\1{2}} $hashes($index) -> d]} {
        if {[regexp {(.)\1{4}} $hashes($index) -> d]} {
#         puts $index
#         puts [llength $cands($d)]
            if {[info exists cands($d)]} {
                foreach idx $cands($d) {
                    if {$index - $idx <= 1000} {
                        lappend keys $idx
    #                     puts "$index: digit $d in $hashes($index) and hashes($idx) = $hashes($idx)"  
                    }
                }
            }
            set cands($d) $index
        } else {
            lappend cands($d) $index
        }
    }

}
puts [lindex [lsort -integer $keys] 63]

18626


3379885 microseconds per iteration

In [5]:
proc stretch {s} {
    time {set s [string tolower [md5::md5 -hex $s]]} 2017
    return $s
}

In [11]:
set seed $input
array unset cands
array unset hashes
set keys {}
array unset stretched

set index 0
for {set index 0} {$index < 50000 } {incr index} {
   if {$index % 100 == 0} {puts $index}
    set hashes($index) [stretch $seed$index]
    if {[regexp {(.)\1{2}} $hashes($index) -> d]} {
        if {[regexp {(.)\1{4}} $hashes($index) -> d]} {
#         puts $index
#         puts [llength $cands($d)]
            if {[info exists cands($d)]} {
                foreach idx $cands($d) {
                    if {$index - $idx <= 1000} {
                        lappend keys $idx
                        puts "$index: digit $d in $hashes($index) and hashes($idx) = $hashes($idx)"  
                    }
                }
            }
            set cands($d) $index
        } else {
            lappend cands($d) $index
        }
    }
}
puts [lindex [lsort -integer $keys] 63]
# parray stretched

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(915) = 6238e1ed738aed24442c46c8c3c8097a
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(1066) = 9cb82444dfa0c60457d2246c7d003610
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(1070) = 65e04b67b511b444d3c89c1c45c602d8
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(1244) = 7403fe823886d4443054f5e16ceb7430
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(1254) = c06f4581bea2ef063e0227ae056ae444
1523: digit 4 in 3594d4c08fe3744444a1b96c66934d52 and hashes(1492) = 080fb579ae088eaa68a131efd4440cbe
1600
1700
1800
1900
2000
2100
2180: digit 3 in 564333331caddf75167399d66e7bcdea and hashes(1272) = 766fa10dec3f44b8c9533372040fc459
2180: digit 3 in 564333331caddf75167399d66e7bcdea and hashes(1630) = 040f8ffe07c59397ee3336e865a2b9db
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
39

eval canceled
    while executing
"puts -nonewline $state(trf) $data"


In [145]:
lindex [lsort -integer $keys] 63

20092

In [107]:
proc stretch {s} {

    set s abc0 ; time {set s [string tolower [md5::md5 -hex $s]]} 2017 ; return $s
}

In [8]:
proc parts input {
    set result1 {}
    set result2 {}

    return [list $result1 $result2]
    
}
aoc::results

Day1	
Day2	
17 microseconds per iteration
